# Dynamic inter-prism calibration results

Notebook which generates the results of the dynamic inter-prism calibration

In [1]:
import numpy as np
import importlib
import sys
sys.path.append("../")
import scripts.theodolite_utils as theodo_u
import scripts.resection_functions as theodo_r
import warnings
warnings.filterwarnings('ignore')
from scipy.spatial.transform import Rotation as R

In [6]:
# This cell gathers result for the inter-prism distance resection
theodo_r = importlib.reload(theodo_r)
theodo_u = importlib.reload(theodo_u)

path = "../data/20221205-3/"
# name_param_file = "f-2-1-1-1-6-0-L_"
name_param_file = "f-2-2-2-3-6-0-L_"
#name_param_file = "ns_sync_p_per_"
robot_type = "Warthog"    # Warthog or HD2 or simulation
debug = True
save_results = True

if robot_type!="simulation":
    i = path+"filtered_prediction/"+name_param_file
    j = path+"total_stations/GCP.txt"
else:
    i = path+"total_stations/"+name_param_file
    j = ""
file_sensors = theodo_u.if_file_exist(path + "sensors_extrinsic_calibration/calibration_results.csv",'')
extrinsic_calibration_results = theodo_u.read_extrinsic_calibration_results_file(file_sensors)
k = extrinsic_calibration_results
if robot_type=="Warthog":
    l = [0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2]
else:
    if robot_type=="HD2":
        l = [0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2]
    else:
        if robot_type=="simulation":
            l = [0.01,1,2,4,7,8,9,10,11,12]
m = ''

Tf_results = []
Error_results = []
Error_results_cp = []
RF = ['','']
print("File: ",i)
for speed in l:
    print("Speed: ", speed)
    dist_error,_,errors_prism,_, errors_exp, TF_list = theodo_r.one_inter_prism_resection_advanced(Inter_distance=k,
                            file_name=i, file_name_marker=j, RF=RF, robot=robot_type, rate=10, prior = "A", velocity_outlier = speed,
                            threshold_training = 0.75, number_iteration = 1, threshold_marker=1, min_6dof=False)
    if len(dist_error)>1:
        RF1 = TF_list[0][1]
        RF2 = TF_list[0][2]
        r1 = R.from_matrix(RF1[0:3,0:3])
        r2 = R.from_matrix(RF2[0:3,0:3])
        euler1 = r1.as_euler('xyz', degrees=True)
        euler2 = r2.as_euler('xyz', degrees=True)
        if debug:
            print(RF1[0:3,3],RF2[0:3,3])
            print(euler1,euler2)
            print("Cp error: ", np.median(errors_prism), np.std(errors_prism))
            print("Inter-prism error: ", np.median(errors_exp), np.std(errors_exp))
        RF = [RF1,RF2]
        Tf_results.append([TF_list[0][1],TF_list[0][2]])
        Error_results.append(np.median(errors_exp))
        Error_results_cp.append(np.median(errors_prism))
    else:
        Tf_results.append(['',''])
        Error_results.append(100000)
        Error_results_cp.append(100000)

arr_error = np.array(Error_results)
index_error = theodo_u.check_if_converge_well(arr_error, Tf_results, debug)

if index_error!=-1:

    errors_linear_dynamic_local = []
    errors_linear_dynamic_cp_local = []
    errors_linear_dynamic_cp_exp_local = []
    Tf_results_bis = []

    Error_results_bis = []
    Error_results_cp_bis = []
    RF = Tf_results[index_error]
    print("File: ",i)
    for speed in l:
        print("Speed: ", speed)
        dist_error,_,errors_prism,_, errors_exp, TF_list = theodo_r.one_inter_prism_resection_advanced(Inter_distance=k,
                                file_name=i, file_name_marker=j, RF=RF, robot=robot_type, rate=10, prior = "A", velocity_outlier = speed,
                                threshold_training = 0.75, number_iteration = 1, threshold_marker=1, min_6dof=False)
        errors_linear_dynamic_local.append(dist_error)
        errors_linear_dynamic_cp_local.append(errors_prism)
        errors_linear_dynamic_cp_exp_local.append(errors_exp)

        if len(dist_error)>1:
            RF1 = TF_list[0][1]
            RF2 = TF_list[0][2]
            r1 = R.from_matrix(RF1[0:3,0:3])
            r2 = R.from_matrix(RF2[0:3,0:3])
            euler1 = r1.as_euler('xyz', degrees=True)
            euler2 = r2.as_euler('xyz', degrees=True)
            if debug:
                print(RF1[0:3,3],RF2[0:3,3])
                print(euler1,euler2)
                print("Cp error: ", np.median(errors_prism), np.std(errors_prism))
                print("Inter-prism error: ", np.median(errors_exp), np.std(errors_exp))
            Tf_results_bis.append([TF_list[0][1],TF_list[0][2]])
            Error_results_bis.append(np.median(errors_exp))
            Error_results_cp_bis.append(np.median(errors_prism))
        else:
            Tf_results_bis.append(['',''])
            Error_results_bis.append(100000)
            Error_results_cp_bis.append(100000)

    arr_error_bis = np.array(Error_results_bis)
    index_error_bis = theodo_u.check_if_converge_well(arr_error_bis, Tf_results_bis, debug)
    print("CP error minimum: ", Error_results_cp_bis[index_error_bis])

    Tf_list_minim = []
    T1 = np.identity(4)
    Tf_list_minim.append(T1.tolist())
    for i in Tf_results_bis[index_error_bis]:
        Tf_list_minim.append(i.tolist())

    if save_results and index_error_bis!=-1:
        theodo_u.save_error_list_to_file(errors_linear_dynamic_cp_local[index_error_bis], path+"list_error/errors_linear_dynamic_cp_4dof.csv")
        theodo_u.save_error_list_to_file(errors_linear_dynamic_cp_exp_local[index_error_bis], path+"list_error/errors_linear_dynamic_cp_experiment_4dof.csv")
        theodo_u.save_tf_list_to_file_multi([Tf_list_minim], path+"list_tf/TF_list_linear_dynamic_4dof.csv")


File:  ../data/20221205-3/filtered_prediction/f-2-2-2-3-6-0-L_
Speed:  0.01
Selected points:  5
Results done !
Speed:  0.05
Selected points:  6
Results done !
Speed:  0.1
Selected points:  6
Results done !
Speed:  0.2
Selected points:  21
4-DOF
Time [s]:  0.13097739219665527
Results done !
[-57.97958272  16.31497936   0.42694755] [-23.03259735  40.95301131   0.25183932]
[   0.           0.        -147.4521867] [   0.            0.         -121.39254501]
Cp error:  45.64290741703388 8.233385714249726
Inter-prism error:  23.32760095719089 34.82018373280096
Speed:  0.3
Selected points:  24
4-DOF
Time [s]:  0.11962485313415527
Results done !
[-57.98671358  16.31423943   0.43828787] [-23.04072824  40.95471823   0.24389582]
[   0.            0.         -147.46261886] [   0.            0.         -121.39652459]
Cp error:  37.63818326437843 7.705973604249337
Inter-prism error:  17.414768358174847 35.04655768874257
Speed:  0.4
Selected points:  25
4-DOF
Time [s]:  0.13289117813110352
Results do

In [ ]:
print(np.array(Tf_results_bis[index_error_bis]))

In [ ]:
T12_perturbation = np.array([[0.1542515, 0.9880316, 0, 10], [-0.9880316, 0.1542515, 0, -5],
                             [0, 0, 1, -3], [0, 0, 0, 1]])
T13_perturbation = np.array([[0.6333192, 0.7738907, 0, -6], [-0.7738907, 0.6333192, 0, 4],
                             [0, 0, 1, 1], [0, 0, 0, 1]])

print(np.linalg.inv(T12_perturbation))
print(np.linalg.inv(T13_perturbation))